In [1]:
# execution time tracking
#!pip install ipython-autotime
%load_ext autotime

time: 206 µs (started: 2025-11-13 13:19:51 +00:00)


In [2]:
#libraries
#!pip install transformers
#!pip install torch
#!pip install accelerate
#!pip install bitsandbytes
#!pip install datasets
#!pip install sentence_transformers

time: 342 µs (started: 2025-11-13 13:21:06 +00:00)


In [3]:
#specific version to avoid error with vector store
#!pip install chromadb==0.5.3

time: 264 µs (started: 2025-11-13 13:21:15 +00:00)


In [4]:
#update transformers
#!pip install -U transformers

time: 265 µs (started: 2025-11-13 13:21:21 +00:00)


In [5]:
#loading the mistral-7b-instruct model and the tokenizer
from torch import cuda, bfloat16
import transformers

model_id = 'mistralai/Mistral-7B-Instruct-v0.3'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# begin initializing HF items, need auth token for these
hf_auth = 'hf_token'
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")
#tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/configuration_auto.py:1317: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

time: 1min 34s (started: 2025-11-13 13:21:23 +00:00)


In [6]:
#sample prompt
prompt = "From the following sentence, please extract the design type of the Quantum Cascade Semiconductor laser device. Print none if the value does not exist in the input text: Employing a scattering-assisted injection and resonant-phonon depopulation method, a QCL operating at 17.3 THz frequency has been successfully developed."
prompt = f"[INST]{prompt} [/INST]"

time: 813 µs (started: 2025-11-13 13:23:03 +00:00)


In [7]:
#view prompt
print(prompt)

[INST]From the following sentence, please extract the design type of the Quantum Cascade Semiconductor laser device. Print none if the value does not exist in the input text: Employing a scattering-assisted injection and resonant-phonon depopulation method, a QCL operating at 17.3 THz frequency has been successfully developed. [/INST]
time: 414 µs (started: 2025-11-13 13:23:08 +00:00)


In [8]:
#sample output with pipeline
from transformers import pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=800,)
result = pipe(prompt)
print(result[0]['generated_text'][len(prompt):])

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 The design type of the Quantum Cascade Semiconductor laser device in the given sentence is not explicitly mentioned. However, the operation is based on a scattering-assisted injection and resonant-phonon depopulation method. This information might help in inferring the design, but a specific design type is not provided in the sentence.
time: 7.82 s (started: 2025-11-13 13:23:14 +00:00)


In [13]:
#mounting drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 981 ms (started: 2025-11-13 13:25:38 +00:00)


In [14]:
#loading training data for providing context
import pandas as pd
train_data_df=pd.read_csv('/content/drive/MyDrive/train_data_split.csv')
## Convert the DataFrame to a format compatible with Hugging Face Datasets
train_data_df['Instruction'] = train_data_df['Instruction']+ ". Input Text: " + train_data_df['input_text'] + "\n\nExtracted Properties:\n\n"+ train_data_df['Output'].astype(str)
train_dataset_dict = {
    'instruction': train_data_df['Instruction'].tolist()
}

time: 595 ms (started: 2025-11-13 13:25:41 +00:00)


In [15]:
#verify train data dictionary
#train_dataset_dict

time: 241 µs (started: 2025-11-13 13:25:52 +00:00)


In [16]:
#creating context document from the list
docs=train_dataset_dict['instruction']

time: 356 µs (started: 2025-11-13 13:25:54 +00:00)


In [17]:
#initializing the embedding model
from sentence_transformers import SentenceTransformer
import chromadb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
import torch
embedding_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2', cache_folder = '/data/base_models')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

time: 7.44 s (started: 2025-11-13 13:26:02 +00:00)


In [18]:
#do the embedding
embeddings = embedding_model.encode(docs)

time: 2.69 s (started: 2025-11-13 13:26:21 +00:00)


In [19]:
#verify the embeddings
embeddings.shape

(1040, 768)

time: 1.66 ms (started: 2025-11-13 13:26:38 +00:00)


In [24]:
#indexing-run this only once
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="q_rag_mistral-7b-instruct")

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


time: 5.08 ms (started: 2025-11-13 13:28:05 +00:00)


In [25]:
#addding the train data
collection.add(
    embeddings = embeddings,
    documents=docs,
    ids= [str(i) for i in range(len(docs))]
)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionAddEvent: capture() takes 1 positional argument but 3 were given


time: 923 ms (started: 2025-11-13 13:28:14 +00:00)


In [26]:
#retrieving
def retrieve_vector_db(query, n_results=3):
    results = collection.query(
    query_embeddings = embedding_model.encode(query).tolist(),
    n_results=n_results
    )
    return results['documents']

time: 533 µs (started: 2025-11-13 13:28:23 +00:00)


In [27]:
#loading test data for model evaluation
import pandas as pd
test_data_path='/content/drive/MyDrive/test_data_split.csv'

time: 409 µs (started: 2025-11-13 13:28:30 +00:00)


In [28]:
#processing the test data to add the queries
#defining the queries for the properties-we need to choose a query based on the input
query_temperature="working temperature of the quantum cascade laser"
query_power= "optical power of the quantum cascade laser"
query_frequency="lasing frequency of the quantum cascade laser"
query_hs="heterostructure of the quantum cascade laser"
query_design_type="design type of the quantum cascade laser"

time: 600 µs (started: 2025-11-13 13:28:35 +00:00)


In [29]:
#checking for the query keywords
def check_text(row, column_name):
    text = row[column_name].lower()
    if "working temperature" in text:
        return query_temperature
    elif "optical power" in text:
        return query_power
    elif "lasing frequency" in text:
        return query_frequency
    elif "query_hs" in text:
        return query_hs
    else:
        return query_design_type

time: 648 µs (started: 2025-11-13 13:28:38 +00:00)


In [30]:
#processing the data
def process_csv(input_filepath, output_filepath, column_name):
    # Load the CSV file
    df = pd.read_csv(input_filepath)

    # Check if the specified column exists
    if column_name in df.columns:
        # Apply the check_text function to the specified column and create a new column with the results
        df['queries'] = df.apply(check_text, axis=1, args=(column_name,))
    else:
        print(f"Column '{column_name}' does not exist in the CSV file.")
        return

    # Save the modified DataFrame to a new CSV file
    df.to_csv(output_filepath, index=False)
    print(f"Processed file saved as '{output_filepath}'.")

time: 795 µs (started: 2025-11-13 13:28:45 +00:00)


In [31]:
#processing the test data
input_csv = test_data_path
output_csv = '/content/drive/MyDrive/processed_test_data.csv'
column_to_check = 'Instruction'
process_csv(input_csv, output_csv, column_to_check)

Processed file saved as '/content/drive/MyDrive/processed_test_data.csv'.
time: 1.64 s (started: 2025-11-13 13:28:52 +00:00)


In [32]:
#loading the processed test data
test_data_df=pd.read_csv('/content/drive/MyDrive/processed_test_data.csv')

time: 5.13 ms (started: 2025-11-13 13:28:55 +00:00)


In [33]:
#verifying the processed data
#test_data_df

time: 257 µs (started: 2025-11-13 13:29:01 +00:00)


In [34]:
#putting the data in some order and renaming the columns
test_data_df=test_data_df[['queries', 'Instruction', 'input_text']]
test_data_df.rename(columns={'Instruction': 'instructions', 'input_text': 'texts'}, inplace=True)

time: 8.03 ms (started: 2025-11-13 13:29:03 +00:00)


In [35]:
#verifying the ordered data
#test_data_df

time: 239 µs (started: 2025-11-13 13:29:09 +00:00)


In [36]:
#we need to generate a list of prompts will be passed to the model.
#This is because we have to parse a prompt that contains the context and input text.
#the query is to fetch the context that will be used in every iteration.
#note that this process might take some time depending on the size of the evaluation dataset.
prompts=[]
for a,b,c in zip(test_data_df['queries'],test_data_df['instructions'],test_data_df['texts']):
    results=retrieve_vector_db(a)
    context=results[0]
    prompt = f'''
[INST]
Problem Definition: Extraction of quantum cascade laser properties from text entails extracting properties from a given text description. This should be done without providing any other additional information or explanations.

Example Sentences containing instructions for extracting properties, the input text and the corresponding extracted properties: {context}

Instruction: {b}

Input Text: {c}

[/INST]
'''
    prompts.append(prompt)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


time: 1.67 s (started: 2025-11-13 13:29:12 +00:00)


In [37]:
#verifying the list of prompts generated for each row in the dataset
#display the prompt
#prompts

time: 262 µs (started: 2025-11-13 13:29:20 +00:00)


In [38]:
#verify the length
print(len(prompts))

130
time: 412 µs (started: 2025-11-13 13:29:22 +00:00)


In [39]:
#we now pass the list of prompts to the model to extract responses in an iterative manner.
#getting the output with transformers pipeline
#import pipeline
from transformers import pipeline
prompts=prompts
responses=[] #list to store the responses from the model.
for prompt in prompts:
  pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=800)
  result = pipe(prompt)
  responses.append(result[0]['generated_text'][len(prompt):])

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 fo

time: 2min 43s (started: 2025-11-13 13:29:29 +00:00)


In [40]:
#number of responses
print(len(responses))

130
time: 513 µs (started: 2025-11-13 13:32:54 +00:00)


In [41]:
#verify the responses
#print responses
responses

['Extracted Properties:\n3.3 THz',
 'Extracted Properties:\n\n10 mW',
 'Extracted Properties: 2.07 THz',
 'Extracted Properties:\n\n190 K (pulsed mode)\n125 K (continuous wave mode)',
 'Extracted Properties:\n\n107 K (in pulsed mode)\n91 K (in continuous wave (cw) mode)',
 'Extracted Properties: MH (Multi-heterostructure)',
 "Extracted Properties: 'Seven-well period composed of In0.6Ga0.4As/InAs'",
 'Extracted Properties: 3.0 THz',
 'Extracted Properties:\n\n185 K',
 'Extracted Properties: 325 K',
 'Extracted Properties: resonant-phonon',
 'Extracted Properties: 195 K',
 'Extracted Properties:\n\n24 mW',
 'Extracted Properties: LO phonon (assuming the design type is related to the phonon mentioned in the text)',
 'Extracted Properties:\n\n150 K (Pulsed Mode)\n110 K (Continuous Wave Mode)',
 'Extracted Properties: LO phonon',
 'Extracted Properties: 113 K (for pulsed mode) and 89 K (for continuous wave (cw) mode)',
 'Extracted Properties: 70 K',
 'Extracted Properties: LO phonon',
 'Ext

time: 2.19 ms (started: 2025-11-13 13:33:22 +00:00)


In [ ]:
#writing the model responses to a csv file
import csv

# Name of the csv file to save the list
file_name = "qcl-RAG-mistral-7b-responses.csv"

# Writing the list to a csv file with one column
with open(file_name, "w", newline="") as file:
    writer = csv.writer(file)
    for item in responses:
        writer.writerow([item])

print("List saved to", file_name)

List saved to qcl-RAG-mistral-7b-responses.csv
